In [7]:
import pandas as pd


def normalize_trade_date(value):
    """Normalize any incoming date to a single string format '%m/%d/%Y'."""
    if value is None:
        return None

    # Try pandas first (handles many formats)
    try:
        dt = pd.to_datetime(value, errors="coerce")
        if not pd.isna(dt):
            return dt.strftime("%m/%d/%Y")
    except Exception:
        pass

    from datetime import datetime
    s = str(value).strip()
    if not s:
        return None

    for fmt in ("%Y-%m-%d", "%d/%m/%Y", "%m/%d/%Y", "%m-%d-%Y"):
        try:
            dt = datetime.strptime(s, fmt)
            return dt.strftime("%m/%d/%Y")
        except ValueError:
            continue

    return None


def generate_trades(csv_file, portfolio_type='simple'):
    """
    Generate trade code from CSV.
    
    Args:
        csv_file: Path to CSV file. It must have:
            - one of: ticker, symbol, tvId, tv_Id
            - side, price, quantity columns
            - optional date/cts/mts column for trade date
        portfolio_type: 'simple' or 'full'
    """
    df = pd.read_csv(csv_file)

    # Build case-insensitive column lookup
    col_map = {str(c).lower(): c for c in df.columns}

    # Detect ticker/symbol column (case-insensitive)
    ticker_candidates = ["ticker", "symbol", "tvid", "tv_id"]
    ticker_key = next((c for c in ticker_candidates if c in col_map), None)
    if ticker_key is None:
        raise ValueError(
            f"CSV must contain one of the ticker columns (case-insensitive): {ticker_candidates}. "
            f"Found columns: {list(df.columns)}"
        )
    ticker_col = col_map[ticker_key]

    # Detect required trade columns (case-insensitive)
    required_cols = {"side": None, "price": None, "quantity": None}
    for key in required_cols.keys():
        if key not in col_map:
            raise ValueError(
                f"CSV must contain a '{key}' column (case-insensitive). Found columns: {list(df.columns)}"
            )
        required_cols[key] = col_map[key]

    side_col = required_cols["side"]
    price_col = required_cols["price"]
    quantity_col = required_cols["quantity"]

    # Detect optional date column (date/cts/mts, case-insensitive)
    date_candidates = ["date", "cts", "mts"]
    date_key = next((c for c in date_candidates if c in col_map), None)
    date_col = col_map[date_key] if date_key is not None else None

    print("# Reset portfolio")
    print("reset_portfolio()\n")
    
    for _, row in df.iterrows():
        ticker = str(row[ticker_col]).lower()
        side = str(row[side_col]).lower()
        price = float(row[price_col])
        qty = abs(float(row[quantity_col]))
        # Optional date column; normalize if present
        trade_date = None
        if date_col is not None:
            trade_date = normalize_trade_date(row[date_col])
        
        if portfolio_type == 'simple':
            if trade_date is not None:
                print(f"add_trade(ticker='{ticker}', side='{side}', price={price}, quantity_buy={qty}, date='{trade_date}')")
            else:
                print(f"add_trade(ticker='{ticker}', side='{side}', price={price}, quantity_buy={qty})")
        else:  # full
            if trade_date is not None:
                print(f"add_trade(ticker='{ticker}', asset_type='Equity', side='{side}', price={price}, quantity_buy={qty}, date='{trade_date}')")
            else:
                print(f"add_trade(ticker='{ticker}', asset_type='Equity', side='{side}', price={price}, quantity_buy={qty})")
    
    print("\n# Display")
    print("df = get_portfolio_df()")
    print("df")

In [8]:
# Example usage for portfolio_simple.ipynb
generate_trades('questdb-query-1764846052060.csv')

# Reset portfolio
reset_portfolio()

add_trade(ticker='eth_crypto', side='buy', price=2797.38187797189, quantity_buy=1.0, date='12/02/2025')
add_trade(ticker='xrp_crypto', side='buy', price=2.00578824078328, quantity_buy=3000.0, date='12/02/2025')
add_trade(ticker='aapl_nasdaq', side='buy', price=282.81, quantity_buy=2.0, date='12/02/2025')
add_trade(ticker='xrp_crypto', side='buy', price=2.04373488487825, quantity_buy=1.0, date='12/02/2025')
add_trade(ticker='aapl_nasdaq', side='buy', price=282.68, quantity_buy=20.0, date='12/02/2025')
add_trade(ticker='sol_crypto', side='buy', price=132.060646499455, quantity_buy=4.0, date='12/02/2025')
add_trade(ticker='doge_crypto', side='buy', price=0.1433955435275, quantity_buy=10000.0, date='12/02/2025')
add_trade(ticker='doge_crypto', side='sell', price=0.150577323494532, quantity_buy=5000.0, date='12/03/2025')
add_trade(ticker='doge_crypto', side='buy', price=0.150144471443905, quantity_buy=3000.0, date='12/03/2025')
add_trade(ticker='doge_cry